In [1]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import requests
import json 
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
# IF YOU ARE SELECTING NIFTY ,BANKNIFTY,MIDCAP  SELECT SEW[0] FOR INDICES ELSE FOR STOCK SELECT SWG[1] 
symbol="NIFTY"
select_swg=["indices","equities"]
seg=select_swg[0]
EX="28-Mar-2024" #enter the expiry date 

In [3]:

chrome_options = Options()


headers = {
    ":authority":"www.nseindia.com",
    ":method":"GET",
    #":path":f"/api/option-chain-equities?symbol={symbol}",
    ":scheme":"https",
    "Accept":"*/*",
    "Accept-Encoding":"gzip, deflate, br" ,
    "Accept-Language":"en-GB,en-US;q=0.9,en;q=0.8,hi;q=0.7",
    "Referer":"https://www.nseindia.com/option-chain",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
    
}

for key, value in headers.items():
    chrome_options.add_argument(f"--header={key}:{value}")


driver = webdriver.Chrome(options=chrome_options)


driver.get("https://www.nseindia.com/option-chain")
driver.get(f"https://www.nseindia.com/api/option-chain-{seg}?symbol={symbol}")
#driver.get(f"https://www.nseindia.com/api/option-chain-equities?symbol={symbol}")
json_content = driver.page_source
soup = BeautifulSoup(json_content, 'html.parser')

In [4]:
pre_tag = soup.find('pre')
if pre_tag:
    json_content = pre_tag.string.strip()
    print("works")
else:
    print("No <pre> tag found in the HTML content.")

works


In [5]:
data_dict = json.loads(json_content)

In [6]:
a=data_dict['records']['data']

In [7]:
ocdata=[]

In [8]:
for i in a:
    for j,k in i.items():
        if j=="CE" or j=="PE":
            info=k
            info["instrumentTyoe"]=j
            ocdata.append(info)


In [23]:
print(len(ocdata))

1363


In [10]:
df=pd.DataFrame(ocdata)
df.drop(columns=['identifier'],inplace=True)

In [11]:
df=df.sort_values(by='expiryDate')

In [12]:
df.reset_index(drop=True,inplace=True)

In [13]:
df

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
0,22300,07-Mar-2024,NIFTY,3193.0,1864,140.255831,9634,13.81,169.60,-97.80,-36.574420,12200,10800,100,167.00,50,173.45,22055.05,CE
1,20050,07-Mar-2024,NIFTY,24.0,0,0.000000,0,0.00,0.00,0.00,0.000000,17300,0,2400,3.05,0,0.00,22055.05,PE
2,22400,07-Mar-2024,NIFTY,992.0,254,34.417344,4859,13.72,136.85,-79.85,-36.848177,7600,8400,900,127.15,1500,136.80,22055.05,CE
3,22400,07-Mar-2024,NIFTY,342.0,114,50.000000,853,15.15,430.25,90.00,26.451139,1900,4000,50,424.25,1400,444.00,22055.05,PE
4,20100,07-Mar-2024,NIFTY,2.0,0,0.000000,0,0.00,0.00,0.00,0.000000,3250,1750,1500,1956.40,1750,2313.05,22055.05,CE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358,15000,31-Dec-2026,NIFTY,44.0,0,0.000000,0,0.00,0.00,0.00,0.000000,850,0,850,23.05,0,0.00,22055.05,PE
1359,14000,31-Dec-2026,NIFTY,19.0,0,0.000000,0,0.00,0.00,0.00,0.000000,1200,0,750,20.95,0,0.00,22055.05,PE
1360,13000,31-Dec-2026,NIFTY,4.0,0,0.000000,0,0.00,0.00,0.00,0.000000,750,0,750,13.55,0,0.00,22055.05,PE
1361,25000,31-Dec-2026,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,600,0,250,675.05,0,0.00,22055.05,PE


In [14]:
df['expiryDate']==EX

0       False
1       False
2       False
3       False
4       False
        ...  
1358    False
1359    False
1360    False
1361    False
1362    False
Name: expiryDate, Length: 1363, dtype: bool

In [15]:
df[df['instrumentTyoe']=='CE']

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
0,22300,07-Mar-2024,NIFTY,3193.0,1864,140.255831,9634,13.81,169.60,-97.80,-36.574420,12200,10800,100,167.00,50,173.45,22055.05,CE
2,22400,07-Mar-2024,NIFTY,992.0,254,34.417344,4859,13.72,136.85,-79.85,-36.848177,7600,8400,900,127.15,1500,136.80,22055.05,CE
4,20100,07-Mar-2024,NIFTY,2.0,0,0.000000,0,0.00,0.00,0.00,0.000000,3250,1750,1500,1956.40,1750,2313.05,22055.05,CE
6,22350,07-Mar-2024,NIFTY,223.0,-2,-0.888889,990,13.67,164.95,-71.90,-30.356766,5750,1650,100,148.60,50,165.00,22055.05,CE
9,20150,07-Mar-2024,NIFTY,0.0,0,0.000000,0,0.00,0.00,0.00,0.000000,3250,1750,1500,1908.10,1750,2259.00,22055.05,CE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319,20900,29-Feb-2024,NIFTY,1952.0,-4,-0.204499,51,21.45,1187.45,-114.20,-8.773480,7500,6450,200,1133.40,1850,1151.95,22055.05,CE
1321,21550,29-Feb-2024,NIFTY,601.0,1,0.166667,438,8.75,530.60,-172.55,-24.539572,12000,10450,200,528.15,50,541.25,22055.05,CE
1324,22800,29-Feb-2024,NIFTY,41924.0,16006,61.756308,131204,12.70,7.95,-16.95,-68.072289,53100,43800,50,7.05,600,7.95,22055.05,CE
1325,19650,29-Feb-2024,NIFTY,4.0,0,0.000000,0,0.00,0.00,0.00,0.000000,6050,6050,1500,2217.95,1500,2501.95,22055.05,CE


In [16]:
df[df['instrumentTyoe']=='PE']

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
1,20050,07-Mar-2024,NIFTY,24.0,0,0.0,0,0.00,0.00,0.0,0.000000,17300,0,2400,3.05,0,0.00,22055.05,PE
3,22400,07-Mar-2024,NIFTY,342.0,114,50.0,853,15.15,430.25,90.0,26.451139,1900,4000,50,424.25,1400,444.00,22055.05,PE
5,20100,07-Mar-2024,NIFTY,20.0,0,0.0,0,0.00,0.00,0.0,0.000000,9250,0,2400,3.05,0,0.00,22055.05,PE
7,22350,07-Mar-2024,NIFTY,106.0,96,960.0,329,13.63,400.35,82.9,26.114349,1950,3250,150,379.30,1800,421.95,22055.05,PE
8,20150,07-Mar-2024,NIFTY,0.0,0,0.0,0,0.00,0.00,0.0,0.000000,9250,0,2400,3.05,0,0.00,22055.05,PE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1357,16000,31-Dec-2026,NIFTY,49.0,0,0.0,0,0.00,0.00,0.0,0.000000,1150,0,800,28.35,0,0.00,22055.05,PE
1358,15000,31-Dec-2026,NIFTY,44.0,0,0.0,0,0.00,0.00,0.0,0.000000,850,0,850,23.05,0,0.00,22055.05,PE
1359,14000,31-Dec-2026,NIFTY,19.0,0,0.0,0,0.00,0.00,0.0,0.000000,1200,0,750,20.95,0,0.00,22055.05,PE
1360,13000,31-Dec-2026,NIFTY,4.0,0,0.0,0,0.00,0.00,0.0,0.000000,750,0,750,13.55,0,0.00,22055.05,PE


In [17]:
ce_data=df[(df['expiryDate']==EX) & (df['instrumentTyoe']=="CE")]

In [18]:
ce_data.reset_index(drop=True,inplace=True)

In [19]:
ce_data.head(5)

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
0,17000,28-Mar-2024,NIFTY,2953.0,33,1.130137,102,0.00,5110.00,-180.85,-3.418165,3900,3650,300,5103.70,500,5138.10,22055.05,CE
1,19600,28-Mar-2024,NIFTY,6.0,0,0.000000,0,0.00,0.00,0.00,0.000000,6050,2750,1500,2436.10,1000,2734.90,22055.05,CE
2,23600,28-Mar-2024,NIFTY,1404.0,473,50.805585,1841,13.44,35.50,-14.15,-28.499496,32700,1450,100,34.95,100,35.50,22055.05,CE
3,18000,28-Mar-2024,NIFTY,13012.0,-72,-0.550290,216,0.00,4153.45,-159.05,-3.688116,4800,5300,950,4127.45,100,4154.95,22055.05,CE
4,19750,28-Mar-2024,NIFTY,1.0,0,0.000000,1,35.36,2640.70,419.50,18.886188,4550,2750,1750,2140.00,1000,2589.90,22055.05,CE


In [20]:
pe_data=ce_data=df[(df['expiryDate']==EX) & (df['instrumentTyoe']=="PE")]

In [21]:
pe_data.reset_index(drop=True,inplace=True)

In [22]:
pe_data.head(5)

,strikePrice,expiryDate,underlying,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentTyoe
0,21300,28-Mar-2024,NIFTY,5969.0,868,17.016271,4593,16.38,122.6,15.65,14.633006,4850,3350,50,121.75,50,123.00,22055.05,PE
1,19600,28-Mar-2024,NIFTY,344.0,0,0.000000,4,16.90,3.1,-1.65,-34.736842,22800,0,3300,8.20,0,0.00,22055.05,PE
2,23600,28-Mar-2024,NIFTY,0.0,0,0.000000,0,0.00,0.0,0.00,0.000000,5150,5150,600,1381.35,600,1464.80,22055.05,PE
3,23000,28-Mar-2024,NIFTY,6593.0,957,16.980128,4166,15.48,902.0,123.85,15.915955,11800,6550,250,898.70,50,913.65,22055.05,PE
4,19800,28-Mar-2024,NIFTY,3288.0,-194,-5.571511,1145,20.01,16.2,1.55,10.580205,2450,4900,100,15.90,250,16.80,22055.05,PE
